In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pydrake.solvers.mathematicalprogram as mp

In [3]:
import numpy as np
import mpc_tools as mpc
import mpc_tools.mpcqp as mqp

In [14]:
m = 1.
l = 1.
g = 10.
A = np.array([
    [0., 1.],
    [g/l, 0.]
])
B = np.array([
    [0.],
    [1/(m*l**2.)]
])
t_s = .1
sys = mpc.DTLinearSystem.from_continuous(t_s, A, B)
N = 1
Q = np.eye(A.shape[0])/100.
R = np.eye(B.shape[1])
x_max = np.array([[np.pi/6.], [np.pi/20./(N*t_s)]])
x_min = -x_max
u_max = np.array([[m*g*l*np.pi/8.]])
u_min = -u_max

In [76]:
factory = mqp.MPCQPFactory(sys, N, Q, R)
factory.add_state_bound(x_max, x_min)
factory.add_input_bound(u_max, u_min)

mpc_qp = factory.assemble()

In [77]:
prog = mp.MathematicalProgram()
x = prog.NewContinuousVariables(2, N, "x")
u = prog.NewContinuousVariables(1, N, "u")
for j in range(N):
    x_next = sys.A.dot(x[:, j]) + sys.B.dot(u[:, j])
    for i in range(x.shape[0]):
        prog.AddLinearConstraint(x_next[i] <= x_max[i])
        prog.AddLinearConstraint(x_next[i] >= x_min[i])
        
    for i in range(u.shape[0]):
        prog.AddLinearConstraint(u[i, j] <= u_max[i])
        prog.AddLinearConstraint(u[i, j] >= u_min[i])
    
    if j < N - 1:
        for i in range(x.shape[0]):
            prog.AddLinearConstraint(x[i, j + 1] == x_next[i])

    prog.AddQuadraticCost(x_next.dot(Q).dot(x_next))
    prog.AddQuadraticCost(u[:, j].dot(R).dot(u[:, j]))
sym_qp = mqp.CanonicalMPCQP.from_mathematicalprogram(prog, u, x)

In [92]:
[tuple(x) for x in mpc_qp.E]

[(0.0, 0.0),
 (0.0, 0.0),
 (-0.99533668671849473, -0.096343428812791745),
 (-0.69382188718187754, -0.71679655464856407),
 (0.99533668671849473, 0.096343428812791745),
 (0.69382188718187754, 0.71679655464856407)]

In [93]:
sorted(range(mpc_qp.E.shape[0]), key=lambda i: tuple(mpc_qp.E[i,:]))

[2, 3, 0, 1, 5, 4]

In [91]:
np.argsort([tuple(x) for x in mpc_qp.E])

array([[0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1]])

In [78]:
mpc_qp.G

array([[ 1.        ],
       [-1.        ],
       [ 0.00477743],
       [ 0.06938219],
       [-0.00477743],
       [-0.06938219]])

In [79]:
mpc_qp.W

array([[ 3.92699082],
       [ 3.92699082],
       [ 0.49614253],
       [ 1.07189836],
       [ 0.49614253],
       [ 1.07189836]])

In [80]:
mpc_qp.E

array([[ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [-0.99533669, -0.09634343],
       [-0.69382189, -0.71679655],
       [ 0.99533669,  0.09634343],
       [ 0.69382189,  0.71679655]])

In [81]:
np.linalg.norm(np.hstack((mpc_qp.E[-2,:], mpc_qp.G[-2])))

0.99999999999999989

In [82]:
sym_qp.G

array([[ 0.00477743],
       [-0.00477743],
       [ 0.06938219],
       [-0.06938219],
       [ 1.        ],
       [-1.        ]])

In [83]:
sym_qp.W

array([[ 0.49614253],
       [ 0.49614253],
       [ 1.07189836],
       [ 1.07189836],
       [ 3.92699082],
       [ 3.92699082]])

In [84]:
sym_qp.E

array([[-0.99533669, -0.09634343],
       [ 0.99533669,  0.09634343],
       [-0.69382189, -0.71679655],
       [ 0.69382189,  0.71679655],
       [-0.        , -0.        ],
       [-0.        , -0.        ]])

In [85]:
mpc_qp.H

array([[ 2.00020726]])

In [86]:
sym_qp.H

array([[ 2.00020726]])

In [87]:
controller = mpc.MPCController(sym_qp, u.shape[0])

In [88]:
controller.compute_explicit_solution()

Computing critical region for the active set []
(1, 1) (6, 1) (6, 1) (6, 2)
Computing critical region for the active set [0]
(1, 1) (6, 1) (6, 1) (6, 2)
Computing critical region for the active set [1]
(1, 1) (6, 1) (6, 1) (6, 2)
Computing critical region for the active set [2]
(1, 1) (6, 1) (6, 1) (6, 2)
Computing critical region for the active set [3]
(1, 1) (6, 1) (6, 1) (6, 2)
LICQ does not hold for the active set [0, 2]
    corrected active set [2]
Computing critical region for the active set [2]
(1, 1) (6, 1) (6, 1) (6, 2)
LICQ does not hold for the active set [0, 3]
    unfeasible critical region detected
LICQ does not hold for the active set [0, 5]
    unfeasible critical region detected
LICQ does not hold for the active set [1, 2]
    unfeasible critical region detected
LICQ does not hold for the active set [1, 3]
    corrected active set [3]
Computing critical region for the active set [3]
(1, 1) (6, 1) (6, 1) (6, 2)
LICQ does not hold for the active set [1, 4]
    unfeasible